# Construction of a base models with whole dataset
- decision tree 
- random forest
- logistic regression

Public Score 0.14431

## 1. Imports 

In [ ]:
## add current directory to system path for relative import of the castom pakage: avp_pckg
import sys
import os

## geting one level above current path 
path_absolute = os.path.dirname(os.getcwd()) 

if path_absolute in sys.path:
    print(f"alrady in sys.path: {path_absolute}")
else:
    sys.path.append(os.path.dirname(os.getcwd()))
    print(f"added to sys.path: {path_absolute}"  )

## for debagging     
# for p in sys.path:
#     print(p)
    
## check import from avp_pckg
from avp_pckg.DataFrame import AvPdataFrame 

In [10]:
import pandas as pd
# import matplotlib.pyplot as plt
# import numpy as np
# import seaborn as sns


from sklearn.pipeline import Pipelines
# from sklearn.impute import SimpleImputer
# from sklearn.preprocessing import OneHotEncoder, StandardScaler
# from sklearn.compose import ColumnTransformer
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report  # f1_score,  accuracy_score, recall_score, precision_score

## castom package 
from avp_pckg.DataFrame import AvPdataFrame 
from avp_pckg.avp_model_selection import cross_validate_pipe 
from avp_pckg.avp_model_selection import plot_scores, print_scores
from avp_pckg.avp_model_selection import PrepareColsBase # PrepareColsTEncoder




In [ ]:
## switch on if modifications in avp_pckg are requared for auto-reload of the pakages
%load_ext autoreload
%autoreload 2

## 2. Load data as train and test sets.


Data types of the coulumns needs to be checked. In current dataset 'WheelTypeID' column have a mixed datatype and should be handled separatrly 

not solved: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas.

In [ ]:
features_train = pd.read_csv(path_absolute + '\\data\\features_train.csv', parse_dates=['PurchDate'], index_col=0)
features_test = pd.read_csv(path_absolute + '\\data\\features_test.csv', parse_dates=['PurchDate'], index_col=0)

target_train = pd.read_csv(path_absolute + '\\data\\target_train.csv', index_col=0)
target_test = pd.read_csv(path_absolute + '\\data\\target_test.csv', index_col=0)

# features_train.loc[:, 'WheelTypeID'] = features_train['WheelTypeID'].astype(str)
features_train.loc[:, 'WheelTypeID'] = features_train['WheelTypeID'].astype(float)
features_train.loc[:, 'WheelTypeID'] = features_train['WheelTypeID'].astype(str)
print(features_train.shape, target_train.shape)
features_train.columns

### 2.1 Select categorical and numerical columns
For the base model all columns are used (exept 'PurchDate': droped in pipeline) 

In [4]:
cols_cat = ['Auction', 'VehicleAge', 'Make', 'Model', 'Trim', 'SubModel', 'WheelType', 'BYRNO', 'VNZIP1', # cols to use
            'Nationality', 'IsOnlineSale', 'Transmission', 'Color', 'TopThreeAmericanName', 'PRIMEUNIT', 'AUCGUART', 'Size', 'VNST', 'VehYear', 'WheelTypeID'] # cols to drop

cols_num = ['MMRAcquisitionAuctionAveragePrice', 'MMRAcquisitionAuctionCleanPrice',
            'MMRAcquisitionRetailAveragePrice', 'MMRAcquisitonRetailCleanPrice',
            'MMRCurrentAuctionAveragePrice', 'MMRCurrentAuctionCleanPrice',
            'MMRCurrentRetailAveragePrice', 'MMRCurrentRetailCleanPrice', 
            'VehOdo', 'VehBCost', 'WarrantyCost']

## 2. Tree-model with whole dataset
cross-validate 

In [ ]:
# execution time= 34 s 
param_name ='max_depth'
param_range = [3, 4, 5, 6, 7, 8, 9, 10, 12, 14, 16, 18,]
score_dict = cross_validate_pipe(X=features_train,
                                y=target_train,
                                cols_cat=cols_cat,
                                cols_num=cols_num,
                                param_name=param_name,
                                param_range= param_range,
                                cv=5, 
                                max_cat=25,
                                estimator_name='tree',
                                n_jobs=-1,
                                )

plot_scores(score_dict, param_name='max_depth tree')
print_scores(score_dict)


## 2. Random Forest-model with whole dataset
cross-validate 

In [ ]:
# execution time= 2m 12s 
param_name ='max_depth'
param_range = [3, 4, 5, 6, 7, 8, 9, 10, 12, 14, 16, 18,]
score_dict = cross_validate_pipe(X=features_train,
                                y=target_train,
                                cols_cat=cols_cat,
                                cols_num=cols_num,
                                param_name=param_name,
                                param_range= param_range,
                                cv=5, 
                                max_cat=25,
                                estimator_name='forest', # 'forest', 'logistic', 'tree',
                                n_jobs=-1,
                                )

plot_scores(score_dict, param_name=param_name)
print_scores(score_dict)

## 2. Logistic Regresion - model with whole dataset
cross-validate 

In [ ]:
# execution time= 15s 
param_name ='C'
param_range = [0.001, 0.002, 0.004, 0.01, 0.02, 0.04, 0.1, 1, 10,]
score_dict = cross_validate_pipe(X=features_train,
                                y=target_train,
                                cols_cat=cols_cat,
                                cols_num=cols_num,
                                param_name=param_name,
                                param_range= param_range,
                                cv=5, 
                                max_cat=25,
                                estimator_name='logistic', # 'forest', 'logistic', 'tree',
                                n_jobs=-1,
                                )

plot_scores(score_dict, param_name=param_name, xlog=True)
print_scores(score_dict)

Cross-Validation
| model | parameter | f1_cv | precision  | recall | f1-score | support |
|---|---|---|---|---|---|---|
| Tree | depth=4 | 0.376 |- |- | - | - |
| Forest | depth=10 |0.382 | - | - |- | - |
| LogReg | C=0.01 | 0.375 | - | - |- | - |


# Predictoins

In [ ]:

### Tree model #########################################################
pipe_tree = Pipeline(steps=[
('base', PrepareColsBase(cols_cat=cols_cat, cols_num=cols_num, max_cat=25).make_pipe()),
('model', DecisionTreeClassifier(class_weight='balanced', random_state=42, max_depth=4))
])

pipe_tree.fit(features_train, target_train['IsBadBuy'].to_numpy())

pred_tree = pipe_tree.predict(features_test)
df_pred_tree = pd.DataFrame(pred_tree, index=features_test.index)
df_pred_tree.columns = ['tree']

report = classification_report(target_test, pred_tree)
print('tree report: \n', report)
print('tree pred.sum():', pred_tree.sum())



In [ ]:

### Forest model #########################################################
pipe_forest = Pipeline(steps=[
('base', PrepareColsBase(cols_cat=cols_cat, cols_num=cols_num, max_cat=25).make_pipe()),
('model', RandomForestClassifier(class_weight='balanced', random_state=42, max_depth=10))
])

pipe_forest.fit(features_train, target_train['IsBadBuy'].to_numpy())
pred_forest = pipe_forest.predict(features_test)
df_pred_forest = pd.DataFrame(pred_forest, index=features_test.index)
df_pred_forest.columns = ['forest']


report = classification_report(target_test, pred_forest)
print('forest report: \n', report)
print('forest pred.sum():', pred_forest.sum())


In [ ]:
### Logistic #########################################################
pipe_reg = Pipeline(steps=[
('preprocessing', PrepareColsBase(cols_cat=cols_cat, cols_num=cols_num, max_cat=25).make_pipe()),
('model', LogisticRegression(class_weight='balanced', random_state=42, C=0.01))
])

pipe_reg.fit(features_train, target_train['IsBadBuy'].to_numpy())
pred_reg = pipe_reg.predict(features_test)
df_pred_reg = pd.DataFrame(pred_reg, index=features_test.index)
df_pred_reg.columns = ['logistic']

report = classification_report(target_test, pred_reg)
print('regression report: \n', report)
print('logistig pred.sum():', pred_reg.sum())



Cross-Validation
| model | parameter | f1_cv | precision  | recall | f1-score | pred.sum() |
|---|---|---|---|---|---|---|
| Tree | depth=4 | 0.376 |0.30 |0.49  |0.37 | 2162 |
| Forest | depth=10 |0.382 | 0.24 |  0.60 |0.35 | 3143 |
| LogReg | C=0.01 | 0.375 | 0.19  | 0.80  |0.30 | 5548 |



## Ensamble

In [ ]:
df_pred = pd.concat([df_pred_tree, df_pred_forest, df_pred_reg], axis=1)
df_pred.head()

In [ ]:
df_pred = pd.concat([df_pred_tree, df_pred_forest, df_pred_reg], axis=1)
df_pred.loc[:, 'sum'] = df_pred['tree'] + df_pred['forest'] + df_pred['logistic']
df_pred.loc[:, 'result1'] = round((df_pred['sum']+1)/3).astype(int)
df_pred.loc[:, 'result2'] = round((df_pred['sum']+0)/3).astype(int)
df_pred.loc[:, 'result3'] = round((df_pred['sum']-1)/3).astype(int)

display(df_pred.head())
print(df_pred.sum())

In [ ]:
report = classification_report(target_test, df_pred['result3'])
print('result3 report: \n', report)

report = classification_report(target_test, df_pred['result2'])
print('result2: \n', report)

report = classification_report(target_test, df_pred['result1'])
print('result1: \n', report)

Cross-Validation
| model | parameter | f1_cv | precision  | recall | f1-score | pred.sum() |
|---|---|---|---|---|---|---|
| Tree | depth=4 | 0.376 |0.30 |0.49  |0.37 | 2162 |
| Forest | depth=10 |0.382 | 0.24 |  0.60 |0.35 | 3143 |
| LogReg | C=0.01 | 0.375 | 0.19  | 0.80  |0.30 | 5548 |
|Ensamble prec| - |- |0.32|0.48|0.38|1909|
|Ensamble recall| - |- |0.18|0.81|0.30|5752|

Cross-Validation original data set
| model | parameter | f1_cv | precision  | recall | f1-score | pred.sum() |
|---|---|---|---|---|---|---|
| Tree | depth=4 | 0.376 | 0.30 | 0.49  | 0.37 | 2162 |
| Forest | depth=10 |0.382 | 0.24 |  0.60 | 0.35 | 3143 |
| LogReg | C=0.01 | 0.375 | 0.19  | 0.80  | 0.30 | 5548 |
|Ensamble| - |- |0.32 | 0.48 | 0.38 | 1909 |

### Conclusion: 
the best results in f1 score were achieved by unanimous vote of models

In [ ]:
## check function for unanimity vote
from avp_pckg.small_functions import unanimity
pred_lst = [df_pred_tree, df_pred_forest, df_pred_reg]
result = unanimity(pred_lst)

report = classification_report(target_test, result)
print('result report: \n', report)

## Prepare workflow for prediction of aim data

original train data contains target column 'IsBadBuy', therefore
- First step is constract similar structure from features_train
- Second step is to build prediction workflow 
- Third step is to check with feature_test the total workflow
- Forth step is to make final prediction on aim data (test.csv)

In [ ]:
## step 1. constract similar structure from features_train
train =  features_train.join(target_train, sort='index')
test = features_test
train.head()

In [67]:
## step 2. constract prediction workflow 

cols_cat = ['Auction', 'VehicleAge', 'Make', 'Model', 'Trim', 'SubModel', 'WheelType', 'BYRNO', 'VNZIP1', # cols to use
            'Nationality', 'IsOnlineSale', 'Transmission', 'Color', 'TopThreeAmericanName', 'PRIMEUNIT', 'AUCGUART', 'Size', 'VNST', 'VehYear', 'WheelTypeID'] # cols to drop

cols_num = ['MMRAcquisitionAuctionAveragePrice', 'MMRAcquisitionAuctionCleanPrice',
            'MMRAcquisitionRetailAveragePrice', 'MMRAcquisitonRetailCleanPrice',
            'MMRCurrentAuctionAveragePrice', 'MMRCurrentAuctionCleanPrice',
            'MMRCurrentRetailAveragePrice', 'MMRCurrentRetailCleanPrice', 
            'VehOdo', 'VehBCost', 'WarrantyCost']

col_target = 'IsBadBuy'

def basis_model_predict(train, test, cols_cat, cols_num, col_target):
    
    target = train['IsBadBuy'].to_numpy()
    features_train = train.drop(columns=[col_target])
    #features_test = test.sort_index()
    features_test = test
    ## preprocessing 
    prepare_pipe = PrepareColsBase(cols_cat=cols_cat, cols_num=cols_num, max_cat=25).make_pipe()
    prepare_pipe.fit(features_train, target)
    data_pipe_train = prepare_pipe.transform(features_train)
    data_pipe_test = prepare_pipe.transform(features_test)

    ## models
    model_tree =  DecisionTreeClassifier(class_weight='balanced', random_state=42, max_depth=4)
    model_forest = RandomForestClassifier(class_weight='balanced', random_state=42, max_depth=10)
    model_logreg = LogisticRegression(class_weight='balanced', random_state=42, C=0.01)


    model_tree.fit(data_pipe_train, target)
    print('model tree fit')
    model_forest.fit(data_pipe_train, target)
    print('model forest fit')
    model_logreg.fit(data_pipe_train, target)
    print('model logistic regression fit')

    pred_tree = pd.DataFrame(model_tree.predict(data_pipe_test), index=features_test.index)
    pred_forest = pd.DataFrame(model_forest.predict(data_pipe_test), index=features_test.index)
    pred_logreg = pd.DataFrame(model_logreg.predict(data_pipe_test), index=features_test.index)


    result = unanimity([pred_tree, pred_forest, pred_logreg])
    df_result = pd.DataFrame(result)
    # print(df_result.columns)
    df_result = df_result.rename(columns={'sum': col_target })
    return df_result 



In [ ]:
result = basis_model_predict(train, test, cols_cat, cols_num, col_target)
result.head()

In [ ]:
## step 3. Check with feature_test the total workflow
report = classification_report(target_test.sort_index(), result)
print('result report: \n', report)

In [ ]:
## step 4. The final prediction on aim data

data_train = pd.read_csv(path_absolute + '\\data\\DontGetKicked\\training.csv', 
                         parse_dates=['PurchDate'], 
                         index_col=0)
data_train.loc[:, 'WheelTypeID'] = data_train['WheelTypeID'].astype(str)
data_train.head()

data_test = pd.read_csv(path_absolute + '\\data\\DontGetKicked\\test.csv', 
                         parse_dates=['PurchDate'], 
                         index_col=0)
data_test.loc[:, 'WheelTypeID'] = data_test['WheelTypeID'].astype(str)
data_test.head()

In [ ]:
pred_aim = basis_model_predict(data_train, data_test, cols_cat, cols_num, col_target)
pred_aim.head()

In [74]:
pred_aim.to_csv(path_absolute + '\\data\\01_Basis_Model_prediction\\basis_model.csv')
